In [90]:
# StandardScaler를 이용해서 서울시 구의 중심의 GPS 값을 정규화 시킨다. 
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

df = pd.read_csv("GPS_of_Seoul_Cities.csv")
sc = StandardScaler()
print(df.head())

      구        X         Y
0   강서구  37.5603  126.8223
1   양천구  37.5267  126.8534
2   구로구  37.4968  126.8566
3  영등포구  37.5235  126.9084
4   금천구  37.4636  126.9005


array([[' 강서구', 37.5603, 126.8223],
       ['양천구', 37.5267, 126.8534]], dtype=object)

In [91]:
# gps의 X값과 Y값을 각각 저장
gps_X = df.iloc[:,1].values
gps_Y = df.iloc[:,2].values
gps_XY = []
for i in range(len(gps_X)):
    tu = (gps_X[i], gps_Y[i])
    gps_XY.append(tu)

sc.fit(gps_XY)
gps_XY_std = sc.transform(gps_XY)
gps_mean = sc.mean_
gps_var = sc.var_
print("평균 : ", gps_mean)
print("표준편차 : ", gps_var)


평균 :  [ 37.554812 126.988204]
표준편차 :  [0.0031548  0.00720859]


In [92]:
# 코사인 유사도를 통한 산책로 유사도 측정
from sklearn.metrics.pairwise import cosine_similarity
"""
walking_Path : DB에 저장된 모든 산책로의 GPS 리스트 -> 한 줄 마다 각각의 산책로의 GPS가 있다고 가정
user_Path : 사용자가 등록하고자 하는 신규 산책로의 GPS 리스트
""" 
walking_Path = pd.read_csv("walking_Path.csv")
user_Path = pd.read_csv("user_Path.csv")


In [93]:
walk = walking_Path.iloc[0]
walk = walk.dropna()
walk_list = walk.values
walk_list = walk_list.reshape(-1, 2)

In [116]:
"""
알고리즘 
1. walking_Path와 user_Path 정규화
2. walking_Path에서 산책로 하나를 꺼내서 user_Path와 코사인 유사도 계산
3. threshold보다 유사도가 높으면 등록 불가
"""
# user_Path 정규화
user = user_Path.iloc[0]
user = user.dropna()
user_list = user.values
user_list = user_list.reshape(-1,2)
user_std = sc.transform(user_list)

# walking_Path 정규화
X, Y = walking_Path.shape
for i in range(X):
    walking = walking_Path.iloc[i]
    walking = walking.dropna()
    walking_list = walking.values
    walking_list = walking_list.reshape(-1,2)
    walking_std = sc.transform(walking_list)
    
    # 유사도 벡터와 점수
    similarity_vector = cosine_similarity(walking_std, user_std)
    similarity_score = similarity_vector.mean()
    print(similarity_score)
    print(cosine_similarity(walking_std, user_std))
    
    # threshold -> 0.975 (나중에 바뀔수도..??)
    threshold = 0.975
    
    # 유사도가 높으면 반복문 멈추고 등록 불가
    if np.any(np.isclose(similarity_vector,1.0)) and similarity_score > threshold :
        print("등록 불가")
        break
    else :
        continue



0.9754459505185422
[[0.98261592 0.9933864  0.97219667]
 [0.91182302 0.9926537  1.        ]]
등록 불가


In [98]:
walking_Path.shape

(4, 26)

False